In [7]:
# Install necessary libraries
!pip install deepface opencv-python-headless mtcnn pandas

In [8]:
!pip install deepface mtcnn openpyxl

import cv2
import numpy as np
import pandas as pd
from mtcnn import MTCNN
from deepface import DeepFace
from collections import defaultdict
import os

# Paths
video_path = "/content/drive/MyDrive/clarifai/input/concentrated students video.mp4"
output_video_path = "/content/drive/MyDrive/clarifai/output/output_resnet_concentrated.mp4"
excel_path = "/content/drive/MyDrive/clarifai/output/detections_resnet_concentrated.xlsx"

# Initialize
detector = MTCNN()
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (w, h))

# Tracking: store last known face centers to assign IDs
next_id = 1
face_centers = {}
tolerance = 50  # pixel distance to match same student

# Data storage
data = []

def assign_id(center):
    global next_id
    for fid, prev_center in face_centers.items():
        if np.linalg.norm(np.array(center) - np.array(prev_center)) < tolerance:
            face_centers[fid] = center
            return fid
    face_centers[next_id] = center
    next_id += 1
    return next_id - 1

def map_emotion(emotion):
    return emotion

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(rgb)

    for face in faces:
        x, y, w_, h_ = face['box']
        x, y = max(0, x), max(0, y)
        face_crop = rgb[y:y+h_, x:x+w_]

        try:
            # DeepFace emotion prediction
            result = DeepFace.analyze(face_crop, actions=['emotion'], enforce_detection=False)
            emotion = result[0]['dominant_emotion']
            precision = result[0]['emotion'][emotion]
        except:
            emotion = "unknown"
            precision = 0

        engagement = map_emotion(emotion)
        center = (x + w_//2, y + h_//2)
        fid = assign_id(center)

        # Draw
        color = (0, 255, 0) if engagement == 'Engaged' else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x + w_, y + h_), color, 2)
        cv2.putText(frame, f"ID:{fid} {engagement}", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        # Log data
        timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
        data.append([timestamp, fid, engagement, emotion, precision])

    out.write(frame)

cap.release()
out.release()

# Save Excel
df = pd.DataFrame(data, columns=['Time (s)', 'Student ID', 'State', 'Emotion', 'Precision'])
df.to_excel(excel_path, index=False)

print(f"Video saved to: {output_video_path}")
print(f"Excel saved to: {excel_path}")


25-09-23 19:59:38 - 🔗 facial_expression_model_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5 to /root/.deepface/weights/facial_expression_model_weights.h5...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:00<00:00, 16.2MB/s]


Video saved to: /content/drive/MyDrive/clarifai/output/output_resnet_concentrated.mp4
Excel saved to: /content/drive/MyDrive/clarifai/output/detections_resnet_concentrated.xlsx


In [9]:
!pip install deepface mtcnn openpyxl

import cv2
import numpy as np
import pandas as pd
from mtcnn import MTCNN
from deepface import DeepFace
from collections import defaultdict
import os
import time

# Paths
video_path = "/content/drive/MyDrive/clarifai/input/concentrated students video.mp4"
output_video_path = "/content/drive/MyDrive/clarifai/output/output_resnet_concentrated.mp4"
excel_path = "/content/drive/MyDrive/clarifai/output/detections_resnet_concentrated.xlsx"

# Initialize
detector = MTCNN()
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (w, h))

# Tracking: store last known face centers to assign IDs
next_id = 1
face_centers = {}
tolerance = 50  # pixel distance to match same student

# Data storage
data = []

def assign_id(center):
    global next_id
    for fid, prev_center in face_centers.items():
        if np.linalg.norm(np.array(center) - np.array(prev_center)) < tolerance:
            face_centers[fid] = center
            return fid
    face_centers[next_id] = center
    next_id += 1
    return next_id - 1

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Start timer for execution time
    start_time = time.time()

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(rgb)

    for face in faces:
        x, y, w_, h_ = face['box']
        x, y = max(0, x), max(0, y)
        face_crop = rgb[y:y+h_, x:x+w_]

        try:
            # DeepFace emotion prediction
            result = DeepFace.analyze(face_crop, actions=['emotion'], enforce_detection=False)
            emotion = result[0]['dominant_emotion']
            precision = result[0]['emotion'][emotion]
        except:
            emotion = "unknown"
            precision = 0

        center = (x + w_//2, y + h_//2)
        fid = assign_id(center)

        # ✅ Green for happy/neutral, Red otherwise
        if emotion in ["happy", "neutral"]:
            color = (0, 255, 0)
        else:
            color = (0, 0, 255)

        cv2.rectangle(frame, (x, y), (x + w_, y + h_), color, 2)
        cv2.putText(frame, f"ID:{fid} {emotion}", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        # Log data with execution time
        timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
        frame_time = time.time() - start_time  # execution time per frame
        data.append([timestamp, fid, emotion, precision, frame_time])

    out.write(frame)

cap.release()
out.release()

# Save Excel with execution time
df = pd.DataFrame(data, columns=['Time (s)', 'Student ID', 'Emotion', 'Precision', 'Frame Execution Time (s)'])
df.to_excel(excel_path, index=False)

print(f"Video saved to: {output_video_path}")
print(f"Excel saved to: {excel_path}")


Video saved to: /content/drive/MyDrive/clarifai/output/output_resnet_concentrated.mp4
Excel saved to: /content/drive/MyDrive/clarifai/output/detections_resnet_concentrated.xlsx


In [10]:
!pip install deepface mtcnn openpyxl

import cv2
import numpy as np
import pandas as pd
from mtcnn import MTCNN
from deepface import DeepFace
import time

# Paths
video_path = "/content/drive/MyDrive/clarifai/input/concentrated students video.mp4"
output_video_path = "/content/drive/MyDrive/clarifai/output/output_resnet_concentrated.mp4"
excel_path = "/content/drive/MyDrive/clarifai/output/detections_resnet_concentrated.xlsx"

# Initialize
detector = MTCNN()
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (w, h))

# Tracking: store last known face centers to assign IDs
next_id = 1
face_centers = {}
tolerance = 50  # pixel distance to match same student

# Data storage
data = []

def assign_id(center):
    global next_id
    for fid, prev_center in face_centers.items():
        if np.linalg.norm(np.array(center) - np.array(prev_center)) < tolerance:
            face_centers[fid] = center
            return fid
    face_centers[next_id] = center
    next_id += 1
    return next_id - 1

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # ✅ Start timer for frame execution
    start_time = time.time()

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(rgb)

    frame_records = []  # store detections for this frame

    for face in faces:
        x, y, w_, h_ = face['box']
        x, y = max(0, x), max(0, y)
        face_crop = rgb[y:y+h_, x:x+w_]

        try:
            # DeepFace emotion prediction
            result = DeepFace.analyze(face_crop, actions=['emotion'], enforce_detection=False)
            emotion = result[0]['dominant_emotion']
            precision = result[0]['emotion'][emotion]
        except:
            emotion = "unknown"
            precision = 0

        center = (x + w_//2, y + h_//2)
        fid = assign_id(center)

        # ✅ Green for happy/neutral, Red otherwise
        color = (0, 255, 0) if emotion in ["happy", "neutral"] else (0, 0, 255)

        cv2.rectangle(frame, (x, y), (x + w_, y + h_), color, 2)
        cv2.putText(frame, f"ID:{fid} {emotion}", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        # Prepare record (frame_time will be added after processing)
        timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
        frame_records.append([timestamp, fid, emotion, precision])

    # ✅ End timer for frame execution
    frame_time = time.time() - start_time

    # Append execution time to all face detections in this frame
    for rec in frame_records:
        rec.append(frame_time)
        data.append(rec)

    out.write(frame)

cap.release()
out.release()

# Save Excel with execution time
df = pd.DataFrame(data, columns=['Time (s)', 'Student ID', 'Emotion', 'Precision', 'Frame Execution Time (s)'])
df.to_excel(excel_path, index=False)

print(f"Video saved to: {output_video_path}")
print(f"Excel saved to: {excel_path}")


Video saved to: /content/drive/MyDrive/clarifai/output/output_resnet_concentrated.mp4
Excel saved to: /content/drive/MyDrive/clarifai/output/detections_resnet_concentrated.xlsx


In [11]:
!pip install deepface mtcnn openpyxl

import cv2
import numpy as np
import pandas as pd
from mtcnn import MTCNN
from deepface import DeepFace
import time

# Paths
video_path = "/content/drive/MyDrive/clarifai/input/concentrated students video.mp4"
output_video_path = "/content/drive/MyDrive/clarifai/output/output_resnet_concentrated_optimized.mp4"
excel_path = "/content/drive/MyDrive/clarifai/output/detections_resnet_concentrated_optimized.xlsx"

# Initialize
detector = MTCNN()
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (w, h))

# Tracking
next_id = 1
face_centers = {}
tolerance = 50  # pixel distance to match same student

# Data storage
data = []

# Parameters for optimization
FRAME_SKIP = 3        # detect every 3rd frame
RESIZE_FACTOR = 0.5   # process smaller image

def assign_id(center):
    global next_id
    for fid, prev_center in face_centers.items():
        if np.linalg.norm(np.array(center) - np.array(prev_center)) < tolerance:
            face_centers[fid] = center
            return fid
    face_centers[next_id] = center
    next_id += 1
    return next_id - 1

frame_index = 0
last_faces = []  # store last detected faces for skipped frames

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    start_time = time.time()  # ✅ Start frame timer
    frame_index += 1
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Run detection only on selected frames
    if frame_index % FRAME_SKIP == 0:
        small = cv2.resize(rgb, (0, 0), fx=RESIZE_FACTOR, fy=RESIZE_FACTOR)
        detected_faces = detector.detect_faces(small)
        last_faces = []
        for face in detected_faces:
            x, y, w_, h_ = face['box']
            # scale back to original size
            x, y, w_, h_ = int(x/RESIZE_FACTOR), int(y/RESIZE_FACTOR), int(w_/RESIZE_FACTOR), int(h_/RESIZE_FACTOR)
            last_faces.append((x, y, w_, h_))
    faces = last_faces  # use cached faces

    frame_records = []

    for (x, y, w_, h_) in faces:
        x, y = max(0, x), max(0, y)
        face_crop = rgb[y:y+h_, x:x+w_]

        try:
            result = DeepFace.analyze(face_crop, actions=['emotion'], enforce_detection=False)
            emotion = result[0]['dominant_emotion']
            precision = result[0]['emotion'][emotion]
        except:
            emotion, precision = "unknown", 0

        center = (x + w_//2, y + h_//2)
        fid = assign_id(center)

        # ✅ Green for happy/neutral, Red otherwise
        color = (0, 255, 0) if emotion in ["happy", "neutral"] else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x + w_, y + h_), color, 2)
        cv2.putText(frame, f"ID:{fid} {emotion}", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
        frame_records.append([timestamp, fid, emotion, precision])

    # ✅ End frame timer
    frame_time = time.time() - start_time
    for rec in frame_records:
        rec.append(frame_time)
        data.append(rec)

    out.write(frame)

cap.release()
out.release()

# Save Excel
df = pd.DataFrame(data, columns=['Time (s)', 'Student ID', 'Emotion', 'Precision', 'Frame Execution Time (s)'])
df.to_excel(excel_path, index=False)

print(f"✅ Optimized video saved to: {output_video_path}")
print(f"✅ Optimized Excel saved to: {excel_path}")


✅ Optimized video saved to: /content/drive/MyDrive/clarifai/output/output_resnet_concentrated_optimized.mp4
✅ Optimized Excel saved to: /content/drive/MyDrive/clarifai/output/detections_resnet_concentrated_optimized.xlsx
